In [ ]:
#######################Task1#############################################################################

In [18]:
!pip install -U langchain-community

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
!pip install chromadb
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 5.3 MB/s eta 0:00:00


In [24]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# ======== 實際要填的模型與檔案路徑 ========
LLM_MODEL = "microsoft/phi-2"
EMBEDDINGS = "sentence-transformers/all-MiniLM-L6-v2"
QUERY = "Who is WANG HSIEN TING?"
CV_FILE = "/content/drive/MyDrive/AI_hw2/resume.pdf"
DB_PATH = "cv_db"

# ========== Step 1: build LLM ==========
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    LLM_MODEL,
    device_map="auto",              # 自動分配 GPU/CPU
    torch_dtype=torch.float16       # 節省記憶體（視你環境而定）
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.0,
    pad_token_id=tokenizer.eos_token_id
)
llm = HuggingFacePipeline(pipeline=pipe)

# ========== Step 2: without RAG ==========
def wo_RAG():
    print("\n🧪 [只用 LLM 回答]：")
    only_llm_response = llm(QUERY)
    print(only_llm_response)

# ========== Step 3: with RAG ==========
def w_RAG():
    # Step 1: Load & Split Resume PDF
    loader = PyPDFLoader(CV_FILE)
    pages = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = splitter.split_documents(pages)

    # Step 2: Embedding + Vector DB
    embedding = HuggingFaceEmbeddings(model_name=EMBEDDINGS)
    vectordb = Chroma.from_documents(docs, embedding, persist_directory=DB_PATH)
    retriever = vectordb.as_retriever(search_kwargs={"k": 3})

    # Step 3: Prompt + QA Chain
    system_prompt = (
        "Use the given context to answer the question. "
        "If you don't know the answer, say you don't know. "
        "Use three sentence maximum and keep the answer concise. "
        "Context: {context}"
    )
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}")
    ])
    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    chain = create_retrieval_chain(retriever, question_answer_chain)

    result = chain.invoke({"input": QUERY})
    print("\n🧠 [使用 RAG 回答]：")
    print(result['answer'])

# ========== Main ==========
if __name__ == '__main__':
    wo_RAG()
    w_RAG()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



🧪 [只用 LLM 回答]：
Who is WANG HSIEN TING?
Wang Hsien Ting is a Taiwanese politician who was born on January 1, 1955. He is currently the Deputy Minister of the Council of Agriculture of the Executive Yuan.

What is Wang Hsien Ting's educational background?
Wang Hsien Ting graduated from National Taiwan University with a degree in agricultural economics.

What positions did Wang Hsien Ting hold before becoming Deputy Minister of the Council of Agriculture?
Wang Hsien Ting served as the Deputy Minister of the Council of Agriculture from 2014 to 2016 and as the Deputy Minister of the Council of Agriculture from 2016 to 2018. He also served as the Deputy Minister of the Council of Agriculture from 2018 to 2020.

What is Wang Hsien Ting's current role?
Wang Hsien Ting is currently the Deputy Minister of the Council of Agriculture of the Executive Yuan.

What is Wang Hsien Ting's political affiliation?
Wang Hsien Ting is a member of the Kuomintang party.

What is Wang Hsien Ting's area of expe

<ipython-input-24-c0f162cc84dd>:52: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=EMBEDDINGS)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



🧠 [使用 RAG 回答]：
System: Use the given context to answer the question. If you don't know the answer, say you don't know. Use three sentence maximum and keep the answer concise. Context: Don· Software Developer · Prague, Czech Republic · .doe@gmail.com · +420 123 456 789
WANG HSIEN TINGRESUME
Status: Developer at The Company, M.Sc. in Computer Science
Skills: Java, C#/.Net, C++, Python, JavaScript, Ruby, Bash, SQL
Interests: Data Warehouses, Data Lakes, Data Analysis, Data Quality
Activities: Hockey, Football, Tennis, Basketball, Reading, Music
Summary
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque gravida ligula sed rhoncus lobortis. Pellen-

Amet eget augue nam quis sapien eget arcu placerat lobortis vivamus maximus elit id
Eget lacus nec dolor sagittis efficitur aliquam nec metus vitae justo auctor sit erat arr
Condimentum vestibulum, nullam vitae cursus erat, praesent hendrerit leo a turpis
’14/01 - ’14/12 Intern The Oldest Company
Quisque in lacus lorem. In vitae f

In [ ]:
#######################Task2#############################################################################

In [43]:
!pip install -q kaggle
from google.colab import files
uploaded = files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle (2).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [44]:
#使用 Kaggle CLI 連到比賽代號
!kaggle competitions download -c ntu-artificial-intelligence-2025-hw-2
!unzip ntu-artificial-intelligence-2025-hw-2.zip

ntu-artificial-intelligence-2025-hw-2.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  ntu-artificial-intelligence-2025-hw-2.zip
replace AI.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: AI.pdf                  
replace HW2_query.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: HW2_query.csv           


In [45]:
# phi4 setting
!pip install flash_attn==2.7.4.post1
!pip install torch==2.6.0
!pip install transformers==4.48.2
!pip install accelerate==1.3.0
!pip install soundfile==0.13.1
!pip install pillow==11.1.0
!pip install scipy==1.15.2
!pip install torchvision==0.21.0
!pip install backoff==2.2.1
!pip install peft==0.13.2
!pip install sentence_transformers
!pip install pdf2image

In [28]:
#提供 PDF 相關的工具
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.7).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [46]:
#處理 slide
import torch
import csv
import numpy as np
from pdf2image import convert_from_path
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM, GenerationConfig
from sentence_transformers import SentenceTransformer
model_path = "microsoft/Phi-4-multimodal-instruct"
# 載入 processor 與 model，
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype='auto',   # 或依需求設定，例如 torch.bfloat16
    _attn_implementation='flash_attention_2',
).cuda()

generation_config = GenerationConfig.from_pretrained(model_path, 'generation_config.json')

/root/.cache/huggingface/modules/transformers_modules/microsoft/Phi-4-multimodal-instruct/0af439b3adb8c23fda473c4f86001dbf9a226021/speech_conformer_encoder.py:2774: FutureWarning: Please specify CheckpointImpl.NO_REENTRANT as CheckpointImpl.REENTRANT will soon be removed as the default and eventually deprecated.
  lambda i: encoder_checkpoint_wrapper(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [47]:
pdf_path = "AI.pdf"
query_csv_path = "HW2_query.csv"
output_csv_path = "HW2_answer.csv"

In [48]:
def caption_page_image(image, previous_image = None):
    # 建立 prompt，要求模型針對圖片產生描述
    user_prompt = '<|user|>'
    assistant_prompt = '<|assistant|>'
    prompt_suffix = '<|end|>'

    prompt = (
            f'{user_prompt}<|image_1|> Please describe the concept, method, and result of this image from a technical perspective. '
            f'If there is a formula, explain its purpose and the meaning of each parameter. '
            f'If it’s an image, clearly describe the objects shown. '
            f'If it’s a flowchart, briefly and clearly explain the process. '
            f'If the slide is blank, simply respond with "empty." '
            f'If there are performance-related metrics, such as a percentage increase or decrease, be sure to highlight them.'
            f'{prompt_suffix}{assistant_prompt}'
        )

    inputs = processor(text=prompt, images=[image], return_tensors='pt').to('cuda:0')
    # 呼叫模型生成文字描述
    generate_ids = model.generate(
        **inputs,
        max_new_tokens=1048,
        generation_config=generation_config,
        num_logits_to_keep=1,
    )
    # 去除前置 input token 部分
    generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]

    # decode 得到文字描述
    response = processor.batch_decode(
        generate_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )[0]
    return response

In [49]:
images = convert_from_path(pdf_path)

In [50]:
# PDF 產生每頁的 caption（圖片描述）
def generate_pdf_captions(pdf_path):
    """
    利用 pdf2image 將 PDF 各頁轉成圖片，再用 caption_page_image 取得描述。
    回傳 list，每個元素為 { "page_number": <int>, "caption": <str> }
    """
    # 轉換 PDF 各頁為圖片（需安裝 pdf2image 且配置 poppler）
    # images = convert_from_path(pdf_path)
    captions = []
    prev_image = None

    for i, image in enumerate(images[0:463], start=0):
        print(f"正在處理第 {i+1} 頁...")

        if prev_image is None:
          caption = caption_page_image(image)
        else:
          caption = caption_page_image(image, previous_image = prev_image)

        print(f"第 {i+1} 頁描述：{caption}")
        captions.append({
            "page_number": i + 1,
            "caption": caption
        })

        prev_image = image

    return captions

In [51]:
captions = generate_pdf_captions(pdf_path)

正在處理第 1 頁...
第 1 頁描述：The image is a slide from a presentation on Artificial Intelligence. The title of the slide is 'Preliminaries and AI Agent Basics'. The slide features a background with a network of interconnected nodes and lines, symbolizing a digital or AI network. On the right side, there is an illustration of a human head with a circuit board inside it, representing an AI agent. The slide also includes the logo of National Taiwan University and the name and email address of Wen-Huang Cheng, who is presumably the presenter.
正在處理第 2 頁...
第 2 頁描述：The image is a cover page for a presentation titled 'SkillMimic: Learning Reusable Basketball Skills from Demonstrations'. The title is displayed at the top in a blue and yellow gradient background. Below the title, there is a large image of a basketball court with a green robot playing basketball. The robot is positioned near the basket, and the court is marked with standard basketball lines. On the right side of the image, there is a lo

In [52]:
!pip install faiss-cpu

In [53]:
import faiss
import numpy as np
def embed_texts(texts, embed_model):
    embeddings = embed_model.encode(texts, show_progress_bar=True)
    return np.array(embeddings).astype("float32")

def build_caption_index(captions, embed_model):
    texts = [item["caption"] for item in captions]
    embeddings = embed_texts(texts, embed_model)
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index, captions

In [54]:
# 讀取查詢 CSV，格式預設：ID, Question
def load_queries(csv_path):
    queries = []
    with open(csv_path, 'r', encoding='utf-8') as fin:
        reader = csv.DictReader(fin)
        for row in reader:
            queries.append({
                "id": row["ID"],
                "question": row["Question"]
            })
    return queries

In [55]:
# query 與描述的向量相似度，找出最佳匹配頁面
def retrieve_best_page(query, index, captions, embed_model, top_k=3):
    # 將 query 轉成向量
    query_embedding = embed_texts([query], embed_model)
    distances, indices = index.search(query_embedding, top_k)
    best_idx = indices[0][0]
    return captions[best_idx]
# 將結果存成 CSV (格式：ID, Answer)
def save_answers_to_csv(results, output_csv_path):
    with open(output_csv_path, 'w', encoding='utf-8', newline='') as fout:
        writer = csv.writer(fout)
        writer.writerow(["ID", "Answer"])
        for res in results:
            writer.writerow([res["id"], res["page_number"]])

In [56]:
# 建立一個簡單的 2 x 384 的 float32 陣列
test_array = np.zeros((2, 384), dtype=np.float32)
test_array = np.ascontiguousarray(test_array)
# 創建 FAISS 索引並加入數據
index = faiss.IndexFlatL2(384)
index.add(test_array)

In [57]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-cos-v1')

In [58]:
index, caption_list = build_caption_index(captions, model)
queries = load_queries(query_csv_path)
results = []
embed_model = model
for query in queries:
    best_match = retrieve_best_page(query["question"], index, caption_list, embed_model, top_k=5)
    results.append({
        "id": query["id"],
        "page_number": best_match["page_number"]
    })
    print(f"Query: {query['question']}")
    print(f"Best Match: Page {best_match['page_number']}")
output_csv_path = "HW2_answer.csv"
save_answers_to_csv(results, output_csv_path)
print("完成 RAG 檢索，儲存於：", output_csv_path)

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: On which page in the document is the reasoning behind upgrading from a wooden pickaxe to a stone pickaxe for improved efficiency most likely discussed, considering game inventory, nearby entities, and contextual factors such as time of day and equipment?
Best Match: Page 97


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: On which page can you find a comparison of two dynamic programming methods for solving Markov Decision Processes (MDPs), focusing on how iterative reward estimation and iterative strategy optimization compute all optimal values while differing in their update processes and policy handling?
Best Match: Page 377


...

Query: On which page is the idea inspired by psychological analogical reasoning—where humans leverage past experiences to address new challenges—most likely explored?
Best Match: Page 194


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: On which page can you find a discussion about learning from experience to identify unfavorable states, using an example like Pacman, where naïve q-learning lacks prior knowledge of specific states?
Best Match: Page 420


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: On which page can you find information about the approach where reasoning involves answer inference conditioned on a rationale, contrasting it with direct prediction methods, and noting a performance drop exceeding 12%?
Best Match: Page 223
完成 RAG 檢索，儲存於： HW2_answer.csv
